In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원 축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth


# 학습 모델 저장을 위한 라이브러리
import pickle

### 데이터 분포 확인(histogram)

In [ ]:
# parquet 파일 데이터를 읽어온다.
df1 = pd.read_parquet('open/train/5.잔액정보/201807_train_잔액정보.parquet')
df2 = pd.read_parquet('open/train/5.잔액정보/201808_train_잔액정보.parquet')
df3 = pd.read_parquet('open/train/5.잔액정보/201809_train_잔액정보.parquet')
df4 = pd.read_parquet('open/train/5.잔액정보/201810_train_잔액정보.parquet')
df5 = pd.read_parquet('open/train/5.잔액정보/201811_train_잔액정보.parquet')
df6 = pd.read_parquet('open/train/5.잔액정보/201812_train_잔액정보.parquet')

In [ ]:
# 월별 데이터 프레임을 합친다.
all_df = pd.concat([df1, df2, df3, df4, df5, df6])
all_df.reset_index(inplace=True, drop=True)
all_df

In [ ]:
import gc
import os

# 저장 폴더 생성 (없으면 새로 생성)
os.makedirs('plots2', exist_ok=True)

# 숫자형 컬럼만 추출
numeric_cols = all_df.select_dtypes(include='number').columns

# 반복해서 히스토그램 그리고 이미지 저장
for idx, col in enumerate(numeric_cols, start=1):
    plt.figure(figsize=(8, 4))
    sns.histplot(data=all_df, x=col, bins=20, kde=False)
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.tight_layout()
    
    # 이미지 파일로 저장
    filename = f"plots2/{idx:02d}_{col}.png"
    plt.savefig(filename)
    
    # 리소스 해제
    plt.clf()
    plt.close()
    gc.collect()

### 데이터 상관관계(heatmap)

In [ ]:
# 데이터 load
df = pd.read_parquet("open/train/6.채널정보/채널정보 월별 합본_전처리.parquet")

# 복사본 생성
df_corr = df.copy()

# Label Encoding
for col in df_corr.select_dtypes(include=['object', 'category']).columns:
    df_corr[col] = LabelEncoder().fit_transform(df_corr[col].astype(str))

# float64 → float32 변환
for col in df_corr.select_dtypes(include='float64').columns:
    df_corr[col] = df_corr[col].astype('float32')

# int64 → int32 변환
for col in df_corr.select_dtypes(include='int64').columns:
    df_corr[col] = df_corr[col].astype('int32')

# 상관관계 행렬 계산
corr_matrix = df_corr.corr()

# 시각화 (옵션)
plt.figure(figsize=(20, 18))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', annot_kws={"size": 5})
plt.title("상관관계 히트맵")
plt.show()

### VIF(메모리 오류 발생)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = df_corr.select_dtypes(include='number').dropna()

X_const = add_constant(X)

# VIF
vif_df = pd.DataFrame()
vif_df["변수명"] = X_const.columns
vif_df["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]

vif_df = vif_df.sort_values("VIF", ascending=False)
print(vif_df)

### 데이터 전처리

In [ ]:
# 결측치가 있는 컬럼만 추출
missing_cols = all_df1.columns[all_df1.isnull().any()]
print(missing_cols)

In [ ]:
drop_list1 = ['카드론잔액_최종경과월', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_해외_B0M',
'연체잔액_할부_해외_B0M','연체잔액_CA_해외_B0M', '연체일자_B0M']

all_df1.drop(drop_list1, axis = 1, inplace = True)

In [ ]:
drop_list2 = ['인입횟수_금융_IB_R6M',
'인입불만횟수_IB_R6M',
'인입불만일수_IB_R6M',
'인입불만월수_IB_R6M',
'인입불만후경과월_IB_R6M',
'인입불만횟수_IB_B0M',
'인입불만일수_IB_B0M',
'IB문의건수_한도_B0M',
'IB문의건수_결제_B0M',
'IB문의건수_할부_B0M',
'IB문의건수_정보변경_B0M',
'IB문의건수_결제일변경_B0M',
'IB문의건수_명세서_B0M',
'IB문의건수_비밀번호_B0M',
'IB문의건수_SMS_B0M',
'IB문의건수_APP_B0M',
'IB문의건수_부대서비스_B0M',
'IB문의건수_포인트_B0M',
'IB문의건수_BL_B0M',
'IB문의건수_분실도난_B0M',
'IB문의건수_CA_B0M',
'IB상담건수_VOC_B0M',
'IB상담건수_VOC민원_B0M',
'IB상담건수_VOC불만_B0M',
'IB상담건수_금감원_B0M',
'IB문의건수_명세서_R6M',
'IB문의건수_APP_R6M',
'IB상담건수_VOC_R6M',
'IB상담건수_VOC민원_R6M',
'IB상담건수_VOC불만_R6M',
'IB상담건수_금감원_R6M',
'불만제기건수_B0M',
'불만제기건수_R12M',
'당사PAY_방문횟수_B0M',
'당사PAY_방문횟수_R6M',
'당사PAY_방문월수_R6M',
'OS구분코드']

all_df2.drop(drop_list2, axis = 1, inplace = True)

In [ ]:
# object형으로 변환하고 싶은 컬럼 리스트
columns_to_convert = ['연체일수_B1M',
'연체일수_B2M',
'연체일수_최근',
'최종연체회차']

# 각 컬럼을 object형으로 변환
all_df1[columns_to_convert] = all_df1[columns_to_convert].astype('object')

# 확인
print(all_df1.dtypes)

In [ ]:
### parquet의 특성으로 인해 타입을 object로 바꿔도 돌아오는 문제 발생 따라서 타입을 아예 str로 변경

In [ ]:
# 명시적으로 문자열(str)로 변환
columns_to_convert = ['연체일수_B1M', '연체일수_B2M', '연체일수_최근', '최종연체회차']
all_df1[columns_to_convert] = all_df1[columns_to_convert].astype(str)

In [ ]:
# str형으로 변환하고 싶은 컬럼 리스트
columns_to_convert = ['인입일수_ARS_B0M',
'IB문의건수_사용승인내역_B0M',
'IB문의건수_선결제_B0M',
'IB문의건수_한도_R6M'
]

# 각 컬럼을 str형으로 변환
all_df2[columns_to_convert] = all_df2[columns_to_convert].astype(str)

In [ ]:
all_df1.to_parquet('5.잔액정보_전처리완.parquet')

In [ ]:
all_df2.to_parquet('6.채널정보_전처리완.parquet')